# Introduction to the Spatially Enabled DataFrame

This notebook will introduce Esri's Spatially Enabled DataFrame structure, which is part of the ArcGIS API for Python. It's an extremely handy tool in certain situations, as it extends the Pandas DataFrame structure to add spatial capabilities. The GeoPandas package provides similar functionality, but it introduces varying levels of frustration depending on the environment you're trying to use it in. The ArcGIS API for Python (along with Pandas and ArcPy) is already conveniently installed in the default ArcGIS Pro Python environment. The Spatially Enabled DataFrame gives us access to some powerful data cleaning and analysis tools. Further, it integrates well with Jupyter Notebooks and gives us some decent visualization options completely outside the confines of ArcGIS Pro.

*Note: For simplicity, this tutorial assumes you have access to the default ArcGIS Pro Python environment. This environment includes the ArcGIS API for Python, ArcPy, and Pandas, all of which we'll make use of. You can, however, use the Spatially Enabled DataFrame with a standard Python installation.  Installation instructions are at: https://developers.arcgis.com/python/guide/install-and-set-up/. You'll also need to install Pandas. Unlike most Esri products, the API for Python is freely available and doesn't require any sort of license. They assume you'll be using this with an ArcGIS Enterprise setup or ArcGIS Online, and they enforce this by requiring either access to ArcPy or authentication to ArcGIS Online or ArcGIS Enterprise. Read/write from/to feature classes is usually done with ArcPy, but it can be done with the pyshp package (shapely and fiona are also required) as long as you are able to authenticate somehow. This makes it possible to run in an environment without access to the typical ArcGIS Python installation.*

## 1 Getting Started

We'll use the os module to deal with some file path stuff. The data_path variable should be set to wherever you've stored the data folder associated with this tutorial. If it's in the same folder as the tutorial notebook file, you shouldn't need to change anything:

In [1]:
import os

data_path = "Data"

The foundation of most scripts that make use of the API is the `GIS` object from the `arcgis.gis` module. *(`import arcgis` would get us access to the whole API for Python, but that's not particularly useful for what we're doing here.)* This object represents a connection to a GIS portal. Using the `GIS()` constructor without passing any parameters into it creates an anonymous login to ArcGIS Online and restricts us to publicly available or local datasets (as long as we've got access to ArcPy as well). We'll need this for data visualization inside the notebook:

In [2]:
from arcgis.gis import GIS

mygis = GIS()

We need to provide some authentication if we want to access resources from a specific data portal. We won't do that here, but plenty of examples can be found in the ArcGIS API for Python documentation: https://developers.arcgis.com/python/

Let's use our `GIS` object to show a simple basemap:

In [3]:
mymap = mygis.map("USA", zoomlevel=2)

mymap.basemap = "streets-vector"

mymap

MapView(layout=Layout(height='400px', width='100%'))

Ok, that's enough setup. Let's move on to...

## 2 Reading and Visualizing Local Geospatial Data

A Spatially Enabled DataFrame adds the `spatial` namespace to the Pandas DataFrame to give it spatial capabilities. The Pandas DataFrame is a great way to handle tabular data using Python, and adding sptial capabilities means that it can make use of any geometries that might be associated with that tabular data. The Spatially Enabled DataFrame allows us to read and write spatial data formats, visualize spatial data, and perform geospatial operations on the data.

*Another Note: Until recently, very similar functionality was available through something called Spatial DataFrames. These are still available for use but are not actively developed and have been effectively replaced by Spatially Enabled DataFrames.*

Let's see what methods and attributes are available in the `spatial` namespace. We'll need to import the Pandas package to do this:

In [4]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

dir(pd.DataFrame.spatial)

['_HASARCPY',
 '_HASSHAPELY',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__feature_set__',
 '__format__',
 '__ge__',
 '__geo_interface__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_renderer',
 '_check_geometry_engine',
 '_data',
 '_index',
 '_kdtree',
 '_meta',
 '_name',
 '_renderer',
 '_repr_svg_',
 '_sfname',
 '_sindex',
 '_sr',
 '_stype',
 '_viz',
 'area',
 'bbox',
 'centroid',
 'distance_matrix',
 'from_df',
 'from_feather',
 'from_featureclass',
 'from_geodataframe',
 'from_layer',
 'from_table',
 'from_xy',
 'full_extent',
 'geometry_type',
 'has_m',
 'has_z',
 'join',
 'length',
 'name',
 'overlay',
 'plot',
 'project',
 'relationship',
 'renderer',
 'sanitize_column_names',
 'select',
 'set_geometry',
 'sindex',


Well, that's a lot of spatial-sounding stuff. Of particular interest, we see can a handful of methods that start with "from_". These allow us to import local data into the Spatially Enabled DataFrame format *(from here on out this will be referred to as SEDF because I don't want to type the full name any more)*. Let's import some data and see how this works:

In [5]:
eqpath = os.path.join(data_path, "earthquakes.shp")

sedf = pd.DataFrame.spatial.from_featureclass(eqpath)

sedf

,FID,Date,Time,Latitude,Longitude,Type,Depth,Depth_Erro,Depth_Seis,Magnitude,...,Azimuthal_,Horizontal,Horizont_1,Root_Mean_,ID,Source,Location_S,Magnitud_2,Status,SHAPE
0,0,1965-01-02,13:44:18,19.2460,145.6160,Earthquake,131.60,,,6.0,...,,,,,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic,"{""x"": 145.6160000000001, ""y"": 19.2460000000000..."
1,1,1965-01-04,11:29:49,1.8630,127.3520,Earthquake,80.00,,,5.8,...,,,,,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic,"{""x"": 127.35200000000009, ""y"": 1.8630000000000..."
2,2,1965-01-05,18:05:58,-20.5790,-173.9720,Earthquake,20.00,,,6.2,...,,,,,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic,"{""x"": -173.97199999999998, ""y"": -20.5789999999..."
3,3,1965-01-08,18:49:43,-59.0760,-23.5570,Earthquake,15.00,,,5.8,...,,,,,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic,"{""x"": -23.55699999999996, ""y"": -59.07599999999..."
4,4,1965-01-09,13:32:50,11.9380,126.4270,Earthquake,15.00,,,5.8,...,,,,,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic,"{""x"": 126.42700000000002, ""y"": 11.938000000000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,23407,2016-12-28,08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40,5.6,...,42.47,0.12,,0.1898,NN00570710,NN,NN,NN,Reviewed,"{""x"": -118.89409999999998, ""y"": 38.39170000000..."
23408,23408,2016-12-28,09:13:47,38.3777,-118.8957,Earthquake,8.80,2,33,5.5,...,48.58,0.129,,0.2187,NN00570744,NN,NN,NN,Reviewed,"{""x"": -118.89569999999998, ""y"": 38.37770000000..."
23409,23409,2016-12-28,12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,,5.9,...,91,0.992,4.8,1.52,US10007NAF,US,US,US,Reviewed,"{""x"": 140.4262000000001, ""y"": 36.9179000000000..."
23410,23410,2016-12-29,22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,,6.3,...,26,3.553,6,1.43,US10007NL0,US,US,US,Reviewed,"{""x"": 118.66390000000001, ""y"": -9.028299999999..."


Let's take a look at the column info for this SEDF:

In [ ]:
sedf.info()

We can see that we've got a geometry column, which is what makes the SEDF special (and spatial). We can also get spatial reference information:

In [ ]:
sedf.spatial.sr

Now that we've got a Spatially Enabled DataFrame set up, we can go ahead and map it out:

In [ ]:
sedf.spatial.plot(mymap)

You may have realized that the layer was added to the map cell up above. We specified that we wanted it to show up in the "mymap" map widget, so it did just that. If we want to add data to a new map, we need to create a new map.

You have also noted that it looked like trash. We can do a few things to fix this. The `plot` method has quite a few optional parameters for specifying how we want our map to look. It uses a mix of esri and matplotlib syntax and the documentation leaves a bit to be desired, but it generally works: 

In [ ]:
eqmap = mygis.map("USA", zoomlevel=3)

sedf.spatial.plot(map_widget=eqmap,
                  renderer_type='c',
                  method='esriClassifyNaturalBreaks',
                  class_count= 10,
                  col='Magnitude',
                  marker_size=8,
                  line_width=0.5,
                  cmap='Reds')

eqmap

That's a bit better. Not great, but we're not here to learn cartography. Let's move on to some slightly more interesting things.  

## 3 Data Cleaning and Analysis

Spatial data visualization inside of a notebook is pretty neat, but it wouldn't be worth much if we couldn't work with the data in a meaningful way. Fortunately, a SEDF is a Pandas DataFrame at its core, so we can do all kinds of useful stuff. The earthquake data above is a bit messy, so let's start by cleaning it up a bit. The index and the FID are columns are duplicates, so let's make the FID column the index:

In [ ]:
sedf.set_index(["FID"], inplace=True)

sedf.head()

We also can't see all the columns in the DataFrame, so let's change that. The code below sets a global option for Pandas, so it will work for all DataFrames displayed from here on out. If you've got a huge number of columns, you might want to be more careful with this:

In [ ]:
pd.set_option('display.max_columns', None)

sedf.head()

Now we can see all the garbage columns that we might want to delete. Let's get rid of everything we don't want:

In [ ]:
sedf.drop(columns=["Depth_Erro", 
                   "Depth_Seis", 
                   "Magnitude_", 
                   "Magnitude1",
                   "Magnitud_1",
                   "Azimuthal_",
                   "Horizontal",
                   "Horizont_1",
                   "Root_Mean_",
                   "Location_S",
                   "Magnitud_2"], inplace=True)

sedf

Ok, that looks much nicer. Let's explore the data a bit. Since this is Pandas under the hood, indexing works just like it does for a regular DataFrame. Here's a single feature:

In [ ]:
sedf[sedf["ID"]=="UW10530748"]

Here's a set of features that match a condition:

In [ ]:
sedf[sedf["Magnitude"]>=8]

Want to know the maximum magnitude value? We can get that as just the number or the whole associated feature:

In [ ]:
maxval = sedf["Magnitude"].max()

maxrow= sedf.loc[sedf["Magnitude"].idxmax()]

print(maxval)
print("---------------")
print(maxrow)

What about plotting? Well, we can use matplotlib for more complex stuff, but a simple histogram can be done without any trouble:

In [ ]:
sedf.hist(column="Magnitude")

Same with scatter plots:

In [ ]:
sedf.plot.scatter(x="Depth", y="Magnitude")

We can get all the unique values in a column:

In [ ]:
sedf["Type"].unique()

Let's check out what's going on with nuclear explosions!

In [ ]:
nukes = sedf[(sedf["Type"]=="Nuclear Explosion") & (sedf["Date"]>='11/09/1989')]

nukes

*We may need to run the code below if the code above doesn't work*

In [ ]:
nukes = sedf["Date"] = pd.to_datetime(sedf["Date"], format="%m/%d/%Y", errors="coerce")

nukes

Looks like there are some similar lats and longs. Let's see who's doing all this testing:

In [ ]:
nukemap = mygis.map(zoomlevel=1)

nukemap.basemap = "national-geographic"

nukes.spatial.plot(map_widget=nukemap,
                  renderer_type='s',
                  marker_size=10,
                  line_width=0.5,
                  cmap='Reds',
                  cstop=50)

nukemap

## 4 Intersection

Let's do some slightly more complex analysis. What if we want a list of all the earthquakes within 100 miles of Japan? We'll start by importing a shapefile of national boundaries:

In [ ]:
country_path = os.path.join(data_path, "World_Countries_Generalized.shp")

countries = pd.DataFrame.spatial.from_featureclass(country_path)

countries.head()

Let's pull out Japan:

In [ ]:
countries[countries["COUNTRY"]=="Japan"]

We can take a look at that feature:

In [ ]:
countries.SHAPE[113]

Yep, that's Japan. Let's do a little buffer trial run:

In [ ]:
countries.SHAPE[113].buffer(distance=100)

Hmm. That's not what we want. We should at least be able see the general shape of the country. Let's see what's going on:

In [ ]:
countries.spatial.sr

Ah, we're in WGS84 and the units are in degrees. The meaning of 100 units in this case is probably ambiguous and definitely not what we want. Let's try bringing it in with a different projection. We'll use UTM 54N (WKID 32654). Probably not perfect, but at least it's close and in meters:

In [ ]:
from arcgis.geometry import SpatialReference

countries_proj = pd.DataFrame.spatial.from_featureclass(country_path, sr=SpatialReference(32654).as_arcpy)

countries_proj[countries_proj["COUNTRY"]=="Japan"]

And the buffer...

In [ ]:
countries_proj.SHAPE[113].buffer(distance=160934)

Much better! Ok, let's make a SDEF that contains only this buffered feature:

In [ ]:
japan = countries_proj[countries_proj["COUNTRY"]=="Japan"].copy()

japan

Now for the buffer. We'll do this by replacing the existing geometry:

In [ ]:
japan["SHAPE"] = japan.SHAPE.geom.buffer(distance=160934)

Checking the shape...

In [ ]:
japan.SHAPE[113]

The last step is to create a sort of query that gets everything that's not outside of our area of interest and apply that query to our earthquake SEDF:

In [ ]:
japan_query = sedf.SHAPE.geom.disjoint(japan.SHAPE[113]) == False

japan_quakes = quakes_proj[japan_query]

japan_quakes

Mapping the results:

In [ ]:
japanmap = mygis.map("Japan", zoomlevel=4)

japan.spatial.plot(map_widget=japanmap)

japan_quakes.spatial.plot(map_widget=japanmap,
                  renderer_type='c',
                  method='esriClassifyNaturalBreaks',
                  class_count= 10,
                  col='Magnitude',
                  marker_size=8,
                  line_width=0.5,
                  cmap='Reds')

japanmap

## 5 Writing SEDF to Feature Class

Now that we've done all this data manipulation, let's save our results back out to a Shapefile:

In [ ]:
outfile = os.path.join(data_path, "japan_quakes.shp")

japan_quakes.spatial.to_featureclass(outfile)

##  6 More Information

This is only a small portion of what can be done with SEDFs and the ArcGIS API for Python. If you want to learn more, the Esri developer documentation (https://developers.arcgis.com/python/) is a good place to start. Even better, they have a GitHub repo with tons of examples (https://github.com/Esri/arcgis-python-api/tree/master). 